# Polytopes

Fix $n \in \mathbb{N}.$ We say a subset $P \subset \mathbb{R}^n$ is a $\textbf{polytope}$ in $\mathbb{R}^n$ if there exist distinct points $\mathbf{v}_1, \ \dots \ , \mathbf{v}_k \in \mathbb{R}^n$ with $k \ge n+1$ such that 
\begin{align*}
    P &=  \left \lbrace \sum_{i=1}^k \lambda_i \mathbf{v}_i: \lambda_1, \ \dots \ , \lambda_k \ge 0, \sum_{i=1}^k \lambda_i \le1 \right \rbrace.
\end{align*}
In words, $P$ is a region enclosed by $n$-dimensional hyperplanes in $\mathbb{R}^n.$ We refer to the points $\mathbf{v}_1, \ \dots \ , \mathbf{v}_k$ as the $\textbf{vertices}$ of $P.$ In addition, the $\textbf{(topological) interior}$ of $P$ is the set
\begin{align*}
 \text{Int}(P) &= \left \lbrace \sum_{i=1}^k \lambda_i \mathbf{v}_i: \lambda_1, \ \dots \ , \lambda_k > 0, \sum_{i=1}^k \lambda_i < 1 \right \rbrace.
\end{align*}

Note that $P$ is necessarily a $\textbf{convex}$ set i.e. for every $\mathbf{p},\mathbf{q} \in P$ and for every $s \in [0,1],$ we have $s \ \mathbf{p} + (1-s) \  \mathbf{q} \in P.$ In words, for every $\mathbf{p},\mathbf{q} \in P,$ the line segment joining $\mathbf{p}, \mathbf{q}$ is a subset of $P.$

## Important Examples 

In the special case $n=2,$ polytopes are precisely $\textbf{convex polygons}$ from basic geometry. A polygon is a region in $\mathbb{R}^2$ enclosed by line segments. Examples of convex polygons include triangles and parallelograms.

<table><tr>
<td>  <img src="Triangle.png" alt="Drawing" style="width: 200px"/> <center> <figcaption> Triangle </figcaption> </center> </td>
<td>  <img src="Parallelogram.png" alt="Drawing" style="width: 200px"/> <center> <figcaption> Parallelogram </figcaption> </center> </td>
</tr></table>


The most important polytope in $n$-dimensions is the simplex. An $\textbf{($n$-dimensional) simplex}$ is a polytope with exactly $n+1$ affinely independent vertices $\mathbf{v}_1, \ \dots \ , \mathbf{v}_{n+1}.$ This means the set $$\left \lbrace \mathbf{v}_{i+1}-\mathbf{v}_1 \right \rbrace_{i=1}^n$$ is linearly independent in $\mathbb{R}^n.$ We associate a simplex to an $n \times n$ matrix $A,$ in which for each $1 \le i \le n,$ the $i$-th row of $A$ is the vector $\mathbf{v}_{i+1}- \mathbf{v}_1$ written as a row vector. The transpose of this matrix is
$$A^T=\begin{bmatrix} \mathbf{v}_{2}-\mathbf{v}_{1} \ | & \ \dots \  & \ | \ \mathbf{v}_{n+1}-\mathbf{v}_{1} \end{bmatrix},$$ where the $i$-th column is the vector $\mathbf{v}_{i+1}- \mathbf{v}_1$ written as a column vector. We call $A^T$ the $\textbf{affine matrix}$ for the simplex. Intuitively, the simplex is the generalization of the triangle in $\mathbb{R}^2.$ Indeed, in the special case $n=2,$ the simplex is the triangle. In the case $n=3,$ the simplex is the tetrahedron. 


Meanwhile, the $\textbf{($n$-dimensional) parallelepiped}$ is a polytope with vertices $$\left \lbrace \mathbf{v}_1 + \sum_{j=1}^n \lambda_j (\mathbf{v}_{j+1}-\mathbf{v}_1) : \  \lambda_1, \ \dots \ , \lambda_n \in \lbrace 0, 1 \rbrace \right \rbrace,$$ where $\mathbf{v}_1, \ \dots \ , \mathbf{v}_{n+1}$ are affinely independent. Intuitively, the parallelepiped is a generalization of the parallelogram in $\mathbb{R}^2.$ Indeed, in the case $n=2,$ the parallelepiped is the parallelogram. Note by elementary combinatorics, a parallelepiped has exactly $2^n$ vertices.

<table><tr>
<td>  <img src="Tetrahedron.png" alt="Drawing" style="width: 200px;"/> <center> <figcaption> 3D Tetrahedron </figcaption> </center>
<td>  <img src="Parallelepiped.png" alt="Drawing" style="width: 200px;"/> <center> <figcaption> 3D Parallelepiped </figcaption> </center>
</td>
</tr></table>

## Problem Statement

Let $P$ be a polytope in $\mathbb{R}^n.$ A classical problem in combinatorial topology with plenty of real world applications is to $\textbf{triangulate } P,$ which means to decompose $P$ into a union of simplices $S_1, \ \dots \ , S_m$ that have pairwise disjoint interiors. It is a well-known theorem in discrete geometry that $\textbf{any polytope can be triangulated}$. For example, the convex polygon with $k$ vertices (with $k \ge 4$) can be triangulated in the following way: choose one particular vertex, and draw a line segment emanating from the chosen vertex to each non-adjacent vertex. Another example is that the $n$-dimensional parallelepiped can be triangulated into $n!$ different simplices.

In this notebook, we triangulate an arbitrary polytope given its vertices. We first obtain subsets of vertices that are affinely independent and hence define simplices that are candidates for triangulation. Given these candidate simplices, we determine the simplices that have pairwise disjoint interiors iteratively. This amounts to solving a linear programming problem numerically.

In [1]:
import numpy as np
import scipy as sc
import scipy.special
import itertools
from joblib import Parallel, delayed

## Determining Candidate Simplices For Triangulation

Let $P$ be a convex polytope with vertices $V= \lbrace \mathbf{v}_1, \ \dots \ , \mathbf{v}_k \rbrace$ where $k \ge n+1.$ We determine simplices who are candidates for triangulation of $P$ in the following way.

* We first obtain all subsets of $V$ with $n+1$ distinct elements. Each such subset is of the form $$\lbrace \mathbf{v}_{i_1}, \ \dots \ , \mathbf{v}_{i_{n+1}} \rbrace,$$ where $1 \le i_1 < \ \dots \ < i_{n+1} \le k.$ By elementary combinatorics, there are $\binom{k}{n+1}$ total such subsets to consider.

* For each subset $\lbrace \mathbf{v}_{i_1}, \ \dots \ , \mathbf{v}_{i_{n+1}} \rbrace \subset V$ from the previous bullet point, we check whether the vertices of the subset form a valid simplex. By the definition of the simplex stated in the introduction, this means we check whether the subset itself is an affinely independent set in $\mathbb{R}^n$ i.e. whether the set of vectors $\lbrace \mathbf{v}_{i_{j+1}} - \mathbf{v}_{i_1} \rbrace_{j=1}^n$ is a linearly independent set in $\mathbb{R}^n.$ By a theorem in linear algebra, this is equivalent to checking the whether 
the corresponding affine matrix $A^T,$ whose $j$-th column is the vector $\mathbf{v}_{i_{j+1}} - \mathbf{v}_{i_1}$ for each $ 1\le j \le n,$ has nonzero determinant.

* If $\lbrace \mathbf{v}_{i_1}, \ \dots \ , \mathbf{v}_{i_{n+1}} \rbrace \subset V$ is affinely independent, we refer to the simplex defined by this set of vertices as a $\textbf{candidate simplex}$ for the triangulation of $P.$ 

In the code cell below, we determine candidate simplices for the triangulation of $P$ based on its vertices. Each candidate simplex is identified via an affinely independent subset of the original set of vertices. We give an example to determine all candidate tetrahedra for the triangulation of the unit cube $I^3$ (whose vertices are elements of the cartesian product $\lbrace 0,1 \rbrace^3.$ Note that there are $\binom{8}{4}=70$ sets of $4$ vertices to consider. But the sets of $4$ vertices that form the cube's boundary square faces are affinely dependent. So are the sets of $4$ vertices that form the cube'sdiagonal cross sections. This would result in considering $70-6-6=58$ candidate tetrahedra. 

In [2]:
def get_affine_matrix(S):
    S=np.array(S)
    A=S[1:]-S[0]
    return A.T

def is_valid_simplex(S):
    A=get_affine_matrix(S)
    return np.linalg.det(A) !=0 

def get_candidate_simplices(V):
    n=len(V[0])
    all_simplices=itertools.combinations(V,n+1)
    candidate_simplices=[S for S in all_simplices if is_valid_simplex(S)]
    return candidate_simplices

def cube(n):
    vertices=[list(x) for x in itertools.product([0,1],repeat=n)]
    return vertices

get_candidate_simplices(cube(3))

[([0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0]),
 ([0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1]),
 ([0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 1, 0]),
 ([0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 1, 1]),
 ([0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 0]),
 ([0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1]),
 ([0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 1, 0]),
 ([0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 1, 1]),
 ([0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 1, 0]),
 ([0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 1, 1]),
 ([0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 1, 0]),
 ([0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 1, 1]),
 ([0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 0, 0]),
 ([0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 0, 1]),
 ([0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 1, 0]),
 ([0, 0, 0], [0, 1, 0], [0, 1, 1], [1, 1, 1]),
 ([0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 1]),
 ([0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1]),
 ([0, 0, 0], [0, 1, 0], [1, 0, 1], [1, 1, 0]),
 ([0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 1, 1]),
 ([0, 0, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1]),
 ([0, 0, 0], 

## Determining Whether Candidate Simplices Have Disjoint Interiors

Let $S$ and $T$ be a candidate simplices with vertices $\lbrace \mathbf{s}_1, \ \dots \ , \mathbf{s}_{n+1} \rbrace$ and $\lbrace \mathbf{t}_1, \ \dots \ , \mathbf{t}_{n+1} \rbrace.$ We determine whether their interiors (as defined in the introduction) are disjoint in the following algorithmic way:

* We sample a large number of random coefficient vectors (according to the uniform $(0,1)$ distribution) of the form $\mathbf{a}=(a_1, \ \dots \ , a_n) \in (0,1)^n$ satisfying $\sum_{i=1}^n a_i < 1.$
* For each $\mathbf{a}$ from the previous bullet point, we consider the affine matrix $A^T$ for $\lbrace \mathbf{s}_1, \ \dots \ , \mathbf{s}_{n+1} \rbrace$ and compute the corresponding point $$\mathbf{p}=\mathbf{s}_1 + A^T \mathbf{a}.$$ Note that $\mathbf{p} \in \text{Int}(S).$ 
* For each $\mathbf{p}$ from the previous bullet point, we consider the affine matrix $B^T$ for $\lbrace \mathbf{t}_1, \ \dots \ , \mathbf{t}_{n+1} \rbrace$ and solve the equation $$\mathbf{p}= \mathbf{t}_1+ B^T \mathbf{b}$$ for $\mathbf{b}.$ Note there will be a unique solution for $\mathbf{b},$ since $\det(B^T) \ne 0.$
* We check if $\mathbf{b}=(b_1, \ \dots \ , b_n)$ from the previous bullet point satisfies each $b_i>0$ and $\sum_{i=1}^n b_i < 1.$  
* If there exists a sampled $\mathbf{a}$ from the first bullet point that yields a corresponding $\mathbf{b}$ satisfying the previous bullet point, then $S$ and $T$ have joint interiors. Otherwise, we declare $S$ and $T$ to have disjoint interiors.  

We implement this algorithm in the code cell below.

In [3]:
def are_valid_coeffs(coeffs):
    for coeff in coeffs:
        if coeff <=0:
            return False
    return np.sum(coeffs) < 1
            

def get_point_in_simplex_from_coeffs(s,coeffs):
    A=get_affine_matrix(s)
    coeffs=np.array(coeffs)
    v1=np.array(s[0])
    p = v1 + A @ coeffs
    return p


def generate_random_points_in_simplex(S,n_samples=1000):
    n=len(S)-1
    coeffs_list=[np.random.uniform(0,1,n) for i in range(n_samples)]
#     coeffs_list=[np.linspace(0,1,n_samples,endpoint=False)]
#     if n>1:
#         for i in range(1,n):
#             coeffs_list.append(np.random.uniform(0,1-np.sum(coeffs_list,axis=0)))
    
#     coeffs_list=np.vstack(coeffs_list).T[1:]
#     x=list(itertools.product(np.linspace(0,1,n_samples,endpoint=False)[1:],repeat=n))
    
    coeffs_list=[coeffs for coeffs in coeffs_list if are_valid_coeffs(coeffs)]
    points=[get_point_in_simplex_from_coeffs(S,coeffs) for coeffs in coeffs_list]
    return points

def is_in_simplex(S,p):
    A=get_affine_matrix(S)
    v1=np.array(S[0])
    p=np.array(p)
    b=p-v1
    x=np.linalg.solve(A,b)
    return are_valid_coeffs(x)


def have_disjoint_interiors(S,T):
    S_points=generate_random_points_in_simplex(S)
    try:
        next(point for point in S_points if is_in_simplex(T,point))
        return False
    except:
        return True


S=[[0,0,0],[1,0,0],[0,1,0],[0,0,1]]
T=[[1,0,0],[0,1,0],[0,0,1],[1,1,1]]
have_disjoint_interiors(S,T)

True

## Determining The List of Candidate Simplices With Pairwise Disjoint Interiors

Let $\lbrace S_1, \ \dots \ , S_p \rbrace$ be the set of all candidate simplices for triangulation of $P.$ We determine the subset $\Delta \subset \lbrace S_1, \ \dots \ , S_p \rbrace$ of candidate simplices that have pairwise disjoint interiors. We do this in an iterative fashion.

* We first let $\tau_1=1$ and initialize $\Delta=\lbrace S_{\tau_1} \rbrace.$
* We define the set $$A_2=\lbrace m > \tau_1: \text{$S_m, S_{\tau_1}$ have pairwise disjoint interiors} \rbrace.$$ If $A_2$ is empty, we terminate the process. Otherwise, if $A_2$ is nonempty, we let $\tau_2=\inf(A_2)$ and update $\Delta=\lbrace S_{\tau_1}, S_{\tau_2} \rbrace.$
* For each $i>2,$ we define $$A_i=\lbrace m > \tau_{i-1}: \text{$S_m, S_{\tau_1}, \ \dots \ , S_{\tau_{i-1}}$ have pairwise disjoint interiors} \rbrace.$$ If $A_i$ is empty, we terminate the process. Otherwise, if $A_i$ is nonempty, we let $\tau_i=\inf(A_i)$ and update $\Delta=\lbrace S_{\tau_1}, \ \dots \ , S_{\tau_{i-1}}, S_{\tau_i} \rbrace.$

We implement this in the code cell below. 

In [4]:
def simplex_is_pairwise_disjoint_to_other_simplices(S, other_simplices):
    try:
        next(T for T in other_simplices if have_disjoint_interiors(S,T) == False)
        return False
    except:
        return True
    
def determine_pairwise_disjoint_simplices(candidate_simplices):
    if candidate_simplices == []:
        return []
    Delta=[candidate_simplices[0]]
    print("ADDED SIMPLEX 1 ...")
    tau=0    
    foundnewsimplex=True
    while foundnewsimplex:
        try:
            tau= next(i for i in range(tau+1, len(candidate_simplices))
                      if simplex_is_pairwise_disjoint_to_other_simplices(candidate_simplices[i], Delta))
            Delta.append(candidate_simplices[tau])
            print("ADDED SIMPLEX {} ...".format(len(Delta)))
        except:
            foundnewsimplex=False
            print("DONE !")
    return Delta
    
def triangulate(V):
    candidate_simplices=get_candidate_simplices(V)
    Delta=determine_pairwise_disjoint_simplices(candidate_simplices)
    return Delta

triangulate(cube(3))

ADDED SIMPLEX 1 ...
ADDED SIMPLEX 2 ...
ADDED SIMPLEX 3 ...
ADDED SIMPLEX 4 ...
ADDED SIMPLEX 5 ...
ADDED SIMPLEX 6 ...
DONE !


[([0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0]),
 ([0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0]),
 ([0, 0, 1], [0, 1, 1], [1, 0, 0], [1, 0, 1]),
 ([0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 1, 0]),
 ([0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0]),
 ([0, 1, 1], [1, 0, 1], [1, 1, 0], [1, 1, 1])]

## Computing A Polytope's Volume

Thanks to our decomposition, we can now easily compute the volume of the polytope $P.$ This is done by computing the volume of each simplex in its triangulation and summing the individual volumes together. The volume of a simplex $S$ spanned by vertices $\mathbf{s}_1, \ \dots \ , \mathbf{s}_{n+1}$ is given by the formula
$$\text{Vol}(S) = \frac{|\det(A)|}{n!},$$ where $A$ is the matrix with $j$-th row $\mathbf{s}_{j+1}-\mathbf{s}_1.$

We implement this below and verify the volume of the $3$-dimensional unit cube is $1$ with this triangulation method.

In [5]:
def volume(V):
    n=len(V[0])
    pairwise_disjoint_simplices=triangulate(V)
    vols=[np.abs(np.linalg.det(get_affine_matrix(S)))/sc.special.factorial(n) for S in pairwise_disjoint_simplices]
    return np.sum(vols)

volume(cube(3))

ADDED SIMPLEX 1 ...
ADDED SIMPLEX 2 ...
ADDED SIMPLEX 3 ...
ADDED SIMPLEX 4 ...
ADDED SIMPLEX 5 ...
ADDED SIMPLEX 6 ...
DONE !


0.9999999999999999

## Triangulating the Calabi Polytope

We consider $\textbf{$n$-dimensional Calabi Polytope},$ whose vertices are
$$V=\left \lbrace (v_1, \ \dots \ , v_n) \in \lbrace 0,1 \rbrace^n: v_j+v_{j+1} \le 1, v_n+v_1 \le 1, \  j \in \lbrace 1, \ \dots \ , n-1 \rbrace \right \rbrace,$$ and if $n$ is odd, we append the vector $(1/2, \ \dots  \ , 1/2) \in \mathbb{R}^n$ to $V.$ In the code cell below, we obtain the vertices of the Calabi Polytope for $n=4$ and verify its volume is $1/6.$

$\textbf{Remark}:$ Beukers, Calabi, and Kolk proved the intriguing infinite series identity
$$\sum_{k \in \mathbb{N}} \frac{(-1)^k}{(2k+1)^n}= \left(\frac{\pi}{2}\right)^n\text{Volume}(\Delta^n)$$ for each $n \in \mathbb{N}.$ Many other authors studied this particular polytope. See the references.

In [6]:
def is_calabi_vertex(binary_vec):
    for i in range(len(binary_vec)-1):
        if binary_vec[i]+binary_vec[i+1] not in [0,1]:
            return False
    return binary_vec[0]+binary_vec[-1] in [0,1]

def calabi_polytope(n):
    vertices=[v for v in cube(n) if is_calabi_vertex(v)]
    if n%2 !=0:
        vertices.append(list(np.repeat(1/2,n)))
    return vertices

volume(calabi_polytope(4))

ADDED SIMPLEX 1 ...
ADDED SIMPLEX 2 ...
ADDED SIMPLEX 3 ...
ADDED SIMPLEX 4 ...
DONE !


0.16666666666666666

# References

* Beukers, Calabi, and Kolk Paper: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.226.4861&rep=rep1&type=pdf
* Noam Elkies Paper: https://www.maa.org/sites/default/files/pdf/news/Elkies.pdf
http://dedekind.mit.edu/~rstan/pubs/pubfiles/66.pdf
* Richard Stanley Paper: http://dedekind.mit.edu/~rstan/pubs/pubfiles/66.pdf
* Zurab Silagadze Papers: https://link.springer.com/content/pdf/10.1007/s12045-015-0241-0.pdf and https://www.degruyter.com/document/doi/10.1515/gmj-2012-0023/html
* Vivek Kaushik and Daniele Ritelli Paper: https://www.ams.org/journals/qam/2018-76-03/S0033-569X-2018-01499-3/S0033-569X-2018-01499-3.pdf